In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, top_k_top_p_filtering
import torch
from torch import nn
from math import pow
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from bitstring import BitArray

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")

In [ ]:
secret_mes = bytes(input("Enter the message you want to hide. Only use ASCII characters"),"ascii")

## Encryption

In [ ]:
key_len = 16
#keygen
key = get_random_bytes(key_len)
cipher = AES.new(key, AES.MODE_GCM)
#encrypt
ciphertext, tag = cipher.encrypt_and_digest(secret_mes)
encode_this = BitArray(cipher.nonce + ciphertext + tag)

In [ ]:
encode_this

## Encode

In [ ]:
result = "Dear audience,"
current_tokens = tokenizer(result, return_tensors="pt")
#print(current_tokens.input_ids)
next_token_logits = model(**current_tokens).logits[:, -1, :]
filtered_next_token_logits = torch.softmax(top_k_top_p_filtering(next_token_logits, top_p=0.5)[0],0)
print(filtered_next_token_logits.shape)
l_index = 0
r_index = len(filtered_next_token_logits)
l_over=0
r_over=0
j=0
while len(filtered_next_token_logits) > 1:
    b = encode_this[j]


    acc = 0
    if not b:
        i = 0
        while acc<0.5:
            acc += filtered_next_token_logits[i]
            i += 1
        
        r_over = acc-0.5
        l_over *= 2
        r_index -= len(filtered_next_token_logits)-i
        filtered_next_token_logits[i-1] -= r_over
        filtered_next_token_logits = 2*filtered_next_token_logits[:i]
    else:
        i = -1
        while acc<0.5:
            acc += filtered_next_token_logits[i]
            i -= 1
        i += 1

        l_over = acc-0.5
        r_over *= 2
        l_index += len(filtered_next_token_logits)+i
        filtered_next_token_logits[i] -= l_over
        filtered_next_token_logits = 2*filtered_next_token_logits[i:] # in the end replace this by a softmax, will be more stable
    if j>20:
        print(filtered_next_token_logits)
    print(float(acc),i,filtered_next_token_logits.shape,j)
    print("lr index", l_index, r_index, "over", float(l_over), float(r_over))
    j+=1


## Decode

## Decryption

In [ ]:
nonce = decrypt_this[:key_len]
ciphertext = decrypt_this[key_len:-key_len]
tag = decrypt_this[-key_len:]

In [ ]:
#decrypt
cipher = AES.new(key, AES.MODE_GCM, nonce)
cipher.decrypt_and_verify(ciphertext, tag)